In [1]:
# Authors: Rachel Dunn, Janson Lin, Anouk de Brouwer
# Date created: Sep 12, 2020

In [2]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# scrape html from website
r = requests.get('https://www.waterlevels.gc.ca/eng/data/table/2020/wlev_sec/7965')
soup = BeautifulSoup(r.text, 'html.parser')
#print(soup)

In [4]:
# the website has a table with data for each month,
# get all tables by looking for the 'width-100' class
tables = soup.find_all(class_='width-100')
print(tables[0]) # print the January table

<table class="width-100">
<caption>January 2020</caption>
<thead>
<tr>
<th>Day</th>
<th>
<span class="wrap-none">Time</span>
</th>
<th>Height<br/>(m)</th>
</tr>
</thead>
<tbody>
<tr>
<td class="align-right">1</td>
<td class="align-right">2:59 AM</td>
<td class="align-right">1.8</td>
</tr>
<tr>
<td class="align-right">1</td>
<td class="align-right">10:22 AM</td>
<td class="align-right">4.9</td>
</tr>
<tr>
<td class="align-right">1</td>
<td class="align-right">5:14 PM</td>
<td class="align-right">3.0</td>
</tr>
<tr>
<td class="align-right">1</td>
<td class="align-right">9:30 PM</td>
<td class="align-right">3.5</td>
</tr>
<tr>
<td class="align-right">2</td>
<td class="align-right">3:36 AM</td>
<td class="align-right">2.2</td>
</tr>
<tr>
<td class="align-right">2</td>
<td class="align-right">10:54 AM</td>
<td class="align-right">4.9</td>
</tr>
<tr>
<td class="align-right">2</td>
<td class="align-right">6:09 PM</td>
<td class="align-right">2.7</td>
</tr>
<tr>
<td class="align-right">2</td>


In [5]:
# loop over tables to get the year, month, day, time and tide height
date = []
height = []
for table in tables:
    
    # get month and year from caption
    month_year = table.find("caption").text.strip()
    [month,year] = month_year.split()
    
    # get all cells by looking for 'align-right' class
    cell = table.find_all(class_="align-right")
    
    # loop over cells in table
    # every 1st cell has the day, every 2nd cell has the time, every 3rd cell has the height 
    for index in range(len(cell)):
        
        # get day
        if ((index % 3) == 0):
            d = cell[index].text.strip()
        
        # get time 
        if ((index % 3) == 1):
            t = cell[index].text.strip()
            
            # paste year, month, day and time together, and append to date list
            ymdt_str = '-'.join([year,month,d,t])
            #ymdt = datetime.strptime(ymdt_str,'%Y-%B-%d-%I:%M %p')
            date.append(ymdt_str)
        
        # get tide height
        if ((index % 3) == 2):
            height.append(cell[index].text.strip())

In [6]:
# create pandas dataframe
tides = pd.DataFrame()
tides['Date'] = pd.to_datetime(date)
tides['Height_m'] = pd.to_numeric(height)
tides.set_index('Date',inplace=True)

In [7]:
# print top 5 rows
tides.head()

,Height_m
Date,
2020-01-01 02:59:00,1.8
2020-01-01 10:22:00,4.9
2020-01-01 17:14:00,3.0
2020-01-01 21:30:00,3.5
2020-01-02 03:36:00,2.2


In [8]:
# print bottom 5 rows
tides.tail()

,Height_m
Date,
2020-12-30 16:15:00,4.5
2020-12-31 00:00:00,0.6
2020-12-31 07:39:00,5.1
2020-12-31 12:54:00,3.9
2020-12-31 16:57:00,4.5
